In this notebook, we will use regression algorithms to predict the number of rented bikes for a given day

### Load data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

seed=99
random.seed(seed)
np.random.seed(seed)

In [2]:
rental = pd.read_csv('data/rental.csv')

#### Preprocessing

Usually, standardization and normalization may be used on the independent variables as some machine learning algorithm is sensitive to the scale and distribution of the data such as SVM and linear regression. Scaling issues may result in features with larger values having more importance than others and ignore the features with lower values, impacting the model performance.

From our EDA phase, we do not see any significant scaling issues, even though some of the features with numerical label encoding do have higher values.

# Predictive modeling

Standardization function is copied from PS4:

In [ ]:
import numpy as np

def standardize(raw_data):
    return ((raw_data - np.mean(raw_data, axis = 0)) / np.std(raw_data, axis = 0))

In [3]:
from sklearn.model_selection import train_test_split

X = rental.loc[:, rental.columns != 'cnt']
y = rental['cnt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X.head()

,instant,temp,atemp,hum,windspeed,season_1,season_2,season_3,season_4,year_0,...,weekday_3,weekday_4,weekday_5,weekday_6,workingday_0,workingday_1,weathersit_1,weathersit_2,weathersit_3,weathersit_4
0,1,0.24,0.2879,0.81,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,2,0.22,0.2727,0.80,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2,3,0.22,0.2727,0.80,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
3,4,0.24,0.2879,0.75,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
4,5,0.24,0.2879,0.75,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0


## Method 1: Multiple Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
score = lin_reg.score(X_train, y_train)
print(f'Predictive score on training data {score:.4f}')

score = lin_reg.score(X_test, y_test)
print(f'Predictive score on test data {score:.4f}')

The linear regression model provides us with a baseline which we can compare our other predictive algorithms on. This model got a score of 0.69 on the training data and 0.67 on the test set.

The score metric returns the R-squared value which describes how much of the variance the model is able to explain. We will use this metric throughout the notebook to compare the models deciding which has the best goodness of fit.

## Method 2. Gradient Boosting

#### Default model

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbm = GradientBoostingRegressor(random_state=seed)

# Fit the model and make prediction
gbm.fit(X_train, y_train)
y_pred = gbm.predict(X_test)

# Evaluate the model
cross_score = np.mean(cross_val_score(gbm, X_train, y_train, cv=10))
train_score = gbm.score(X_train, y_train)
test_score = gbm.score(X_test, y_test)


print('Training set {:.4f}'.format(train_score))
print('Cross-validated set: {:.4f}'.format(cross_score))
print('Test set: {:.4f}'.format(test_score))

#### Finetuned model
Use a grid search algorithm to find the best hyperparameters for the gradient boosting algorithm.

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [2, 5, 6, 7, 9, 11],
    'n_estimators': [10, 50, 100],
    'min_samples_leaf': [1, 3, 5, 7, 9]
}

gbm = GradientBoostingRegressor(random_state=seed)

cv_search = GridSearchCV(gbm, param_grid, cv=5, verbose=True, n_jobs=-1).fit(X_train, y_train)
print("best parameters:", cv_search.best_params_)
# best parameters: {'learning_rate': 0.1, 'max_depth': 11, 'min_samples_leaf': 7, 'n_estimators': 100}

In [ ]:
print('Training set {:.4f}'.format(cv_search.best_estimator_.fit(X_train, y_train).score(X_train, y_train)))
print('Cross-validated set: {:.4f}'.format(cv_search.best_score_))
print('Test set: {:.4f}'.format(cv_search.best_estimator_.score(X_test, y_test)))

The first model was trained using the default parameters of the gradient boosting algorithm and got a significant higher score than the multiple regression model with an increase from 0.68 to 0.79.

Due to the performance of the gradient boosting algorithm, we decided to finetune the hyperparameters to see if we could get a better performance. After using grid search to test out combinations of different parameters we got a satisfying result of 0.9840 on the training set, 0.9368 on cross-validation, and 0.9422 on the test set with the following parameters:  
* best parameters: {'learning_rate': 0.1, 'max_depth': 11, 'min_samples_leaf': 9, 'n_estimators': 100}

## Method 3. Decision Trees

#### Default model

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor(random_state=seed)

# Fit the model and make prediction
dtr.fit(X_train, y_train)

# Evaluate the model
cross_score = np.mean(cross_val_score(dtr, X_train, y_train, cv=10))
train_score = dtr.score(X_train, y_train)
test_score = dtr.score(X_test, y_test)

print('Training set {:.4f}'.format(train_score)) 
print('Cross-validated score: {:.4f}'.format(cross_score.mean()))
print('Test set {:.4f}'.format(test_score))

The decision tree model with default parameters got a better score than the default gradient boosting alternative, but performed worse than the fine-tuned gradient boosting model. The performance on the training set is also significant better than on the test set. This difference may indicate an overfit on the training set.

#### Fine-tuning of the decision tree

In [ ]:
param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [2, 5, 6, 7, 9, 11],
    'n_estimators': [10, 50, 100],
    'min_samples_leaf': [1, 3, 5, 7, 9]
}

dt = DecisionTreeRegressor(random_state=seed)

cv_search = GridSearchCV(dt, param_grid, cv=5, verbose=True, n_jobs=-1).fit(X_train, y_train)
print("best parameters:", cv_search.best_params_)

In [ ]:
print('Training set {:.4f}'.format(cv_search.best_estimator_.fit(X_train, y_train).score(X_train, y_train)))
print('Cross-validated score: {:.4f}'.format(cv_search.best_score_))
print('Test set: {:.4f}'.format(cv_search.best_estimator_.score(X_test, y_test)))

In [ ]:
from sklearn.tree import plot_tree
import seaborn as sns

# Plot the decision tree
fig = plt.figure(figsize=(32,15))
plot_tree(dtr, fontsize=15, max_depth=3, feature_names=X_train.columns, class_names=['Not survived', 'Survived'], filled=True)
plt.show()

Feature importance

In [ ]:
fig = plt.figure(figsize=(5,5))
sns.barplot(x=dtr.feature_importances_, y=X_train.columns)

## Method 4. Random Forest

In [4]:
from sklearn.ensemble import RandomForestRegressor

# Fit the model and make prediction
rf = RandomForestRegressor(random_state=seed) # Default parameters
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

# Evaluate the model
cross_score = np.mean(cross_val_score(rf, X_train, y_train, cv=10))
train_score = rf.score(X_train, y_train)
test_score = rf.score(X_test, y_test)

print('Training set {:.4f}'.format(train_score))
print('Cross-validated score: {:.4f}'.format(cross_score))
print('Test set: {:.4f}'.format(test_score))

Training set 0.9901
Cross-validated score: 0.9261
Test set: 0.9335


From the default models we earlier have fitted, this gives the best goodness of fit. Therefore, I will fine-tune the parameters in order to see if we can improve the score.

In [ ]:
print('test')

In [6]:
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 5, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


cv_search = GridSearchCV(rf, param_grid, cv=5, verbose=True, n_jobs=-1).fit(X_train, y_train)
print("best parameters:", cv_search.best_params_)
#best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

Fitting 5 folds for each of 135 candidates, totalling 675 fits
best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [8]:
print('Training set {:.4f}'.format(cv_search.best_estimator_.fit(X_train, y_train).score(X_train, y_train)))
print('Cross-validated score: {:.4f}'.format(cv_search.best_score_))
print('Test set: {:.4f}'.format(cv_search.best_estimator_.score(X_test, y_test)))

Training set 0.9901
Cross-validated score: 0.9219
Test set: 0.9335


In [15]:
rf2 = RandomForestRegressor(random_state=seed, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100)
rf2.fit(X_train, y_train)
rf2.score(X_test, y_test)

0.933511308196414

The fine-tuned random forest model

## Method 7. Support Vector Machine

In [ ]:
from sklearn.svm import SVR

# Fine-tune
svr = SVR()
param_grid = {
    'kernel': ['linear', 'poly', 'rbf'],
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 1]
}

#grid_search = GridSearchCV(svr, param_grid, cv=5, verbose=1, n_jobs=-1)
#grid_search.fit(X_train, y_train)
svr.fit(X_train, y_train)

# Evaluate model
cross_score = np.mean(cross_val_score(svr, X_train, y_train, cv=10))
train_score = svr.score(X_train, y_train)
test_score = svr.score(X_test, y_test)

print('Training set {:.4f}'.format(train_score))
print('Cross-validated score: {:.4f}'.format(cross_score))
print('Test set: {:.4f}'.format(test_score))

# Took too long time

Fitting 5 folds for each of 27 candidates, totalling 135 fits


## Method 8. Neural Networks

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1])))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')